## Description of the dataset

"The data set represents 10 years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. It includes over 50 features representing patient and hospital outcomes. Information was extracted from the database for encounters that satisfied the following criteria.

It is an inpatient encounter (a hospital admission). It is a diabetic encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis. The length of stay was at least 1 day and at most 14 days. Laboratory tests were performed during the encounter. Medications were administered during the encounter. The data contains such attributes as patient number, race, gender, age, admission type, time in hospital, medical specialty of admitting physician, number of lab test performed, HbA1c test result, diagnosis, number of medication, diabetic medications, number of outpatient, inpatient, and emergency visits in the year before the hospitalization, etc."



### Target

The target variable is predicting whether a patient after treatment will be readmitted or not. If readmission is predicted then whether the readmission occurs in less than 30 days or wheter it will happen in more than 30 days.

We'll start with some initial observations and continue by addressing the different parts of ML challenge.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('diabetic_data.csv')

In [3]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [4]:
df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


From the description.pdf file attached with the dataset we learn that encounter_id and patient_nbr are specific to every patient and as such are irrelevant from a prediction perspective. As such these columns will be dropped in the next step.

In [5]:
df = df.drop(['encounter_id', 'patient_nbr'], axis=1)

In [6]:
((df['weight'] == '?').sum())/df.shape[0]

0.9685847925633315

Also, the weight column is missing in 96.85%(see above) of the rows. If we keep weight column we risk losing performance because of regularization and/or overfitting in the patients which have weight information. As such the weight column is dropped aswell at this point.

In [7]:
df = df.drop('weight', axis=1)

In [8]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,?,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),1,1,7,3,?,?,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),1,1,7,2,?,?,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),1,1,7,2,?,?,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,1,7,1,?,?,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


From the included pdf, payer_code refers to the mode of payment(If insurance then company or self) for the hospital visit. As such, it's highly likely that this column does not contribute to the target variable and I drop this column aswell.

In [9]:
df = df.drop('payer_code', axis=1)

Now, we will convert all categorical features to their one-hot encoded equivalents. All variables except time_in_hospital and those prefixed with num are categorical. 

In [10]:
df.readmitted.value_counts()

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64

In [11]:
54864/(54864+35545+11357)

0.5391191557101586

## Metric Selection

Even though the classes are skewed, the skew is not so bad that we can't use accuracy. So, we will use accuracy as a primary measure for performance in this project. At times, we will also use the confusion matrix for reference.

In [12]:
cat_columns = []
num_columns = []
for i,c in enumerate(df.columns):
    if not(c=='time_in_hospital' or ('num' in str(c)) or 'readmitted'==c):
        cat_columns.append(c)
    elif c!= 'readmitted':
        num_columns.append(c)

In [13]:
num_columns

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

## Model Selection

I'll be using an ensemble of gradient boosting and random forest classifiers. I'm doing this because most of the data contains categorical features which are handled much better by tree based models. Also these models are usually among the best performing for any kind of structured data project. Finally, these models can also be used to interpret feature importance and help in feature engineering. 

In [14]:
import h2o

In [15]:
h2o.init(max_mem_size = "6G", min_mem_size='2G')             #specify max number of bytes. uses all cores by default.
h2o.remove_all()                          #clean slate, in case cluster was already running

from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\archi\AppData\Local\Temp\tmpuiztlqz7
  JVM stdout: C:\Users\archi\AppData\Local\Temp\tmpuiztlqz7\h2o_archi_started_from_python.out
  JVM stderr: C:\Users\archi\AppData\Local\Temp\tmpuiztlqz7\h2o_archi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_archi_9a0wbc
H2O cluster total nodes:,1
H2O cluster free memory:,5.333 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [16]:

hf = h2o.H2OFrame(df)
for c in hf.types:
    if (not 'num' in c) and (not 'time' in c):
        hf[c] = hf[c].ascharacter().asfactor()
covtype_X = hf.col_names[:-1]     #last column is Cover_Type, our desired response variable 
covtype_y = hf.col_names[-1]    

train, valid, test = hf.split_frame([0.8, 0.1], seed=1234)


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
from h2o.grid.grid_search import H2OGridSearch

gbm_params1 = {'learn_rate': [0.01, 0.1],
                'max_depth': [5, 8, 11],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0],
                'ntrees': [20,50,70]}

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(covtype_X, covtype_y,
                training_frame=train,
                validation_frame=valid,
                seed=1)



gbm Grid Build progress: |██████████████████████

In [ ]:
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='accuracy', decreasing=True)
gbm_gridperf1

In [ ]:
rf_params2 = {
                'max_depth': [20, 35, 50, 75],
                'sample_rate': [0.8, 1.0, 0.6],
                'ntrees': [50, 100, 150, 200]}

# Train and validate a cartesian grid of GBMs
rf_grid2 = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='rf_grid2',
                          hyper_params=rf_params2)
rf_grid2.train(covtype_X, covtype_y,
                training_frame=train,
                validation_frame=valid,
                seed=1)



In [ ]:
rf_gridperf2 = rf_grid2.get_grid(sort_by='accuracy', decreasing=True)


In [ ]:
rf_gridperf2

In [ ]:
best_rf = rf_gridperf2.models[0]
best_gbm = gbm_gridperf1.models[0]

In [ ]:
h2o.save_model(best_rf, 'rf.h5', force=True)
h2o.save_model(best_gbm, 'gbm.h5', force=True)

In [ ]:
best_rf

In [ ]:
best_gbm

The models seem to be overfitting very badly. We will investigate this later.

In [ ]:
pred_gbm = best_gbm.predict(valid)
pred_rf = best_rf.predict(valid)

In [ ]:
td = valid.as_data_frame()
td2 = valid.as_data_frame()

In [ ]:
for i in range(0,21):
    pred = i/20*pred_gbm + (1-i/20)*pred_rf
    pred1= pred.as_data_frame()
    p1 = pred1.iloc[:, 1:]
    td.iloc[td.index[p1['NO'] == np.amax(p1.values, axis=1)],-1] = 'NO'
    td.iloc[td.index[p1['>30'] == np.amax(p1.values, axis=1)],-1] = '>30'
    td.iloc[td.index[p1['<30'] == np.amax(p1.values, axis=1)],-1] = '<30'
    print((((td2 == td)['readmitted']).sum())/(td2['readmitted'].count()), i)

## Feature Engineering and Selection

Let's look at feature importance first up

In [ ]:
best_gbm.varimp_plot()

The diagnoses appear to be very important, however there are 900+ unique values for each which leads to massive overfit potential. Let's try without these.

In [ ]:
best_gbm.train(list(set(train.col_names) - set(['readmitted','diag_1', 'diag_2', 'diag_3'])), covtype_y,
                training_frame=train,
                validation_frame=valid)


In [ ]:
best_gbm

In [ ]:
best_gbm.varimp_plot()

Expectedly, our error on validation set decreases.

## Feature Engineering

Create feature from combination of diagnoses and replace the individual diagnoses with their target mean

In [ ]:
interaction_cols1 = ['diag_3', 'diag_2', 'diag_1']

train_cols = train.interaction(factors=interaction_cols1,    #Generate pairwise columns
                               pairwise=False,
                               max_factors=1000,
                               min_occurrence=100)
valid_cols = valid.interaction(factors=interaction_cols1,
                               pairwise=False,
                               max_factors=1000,
                               min_occurrence=100)
test_cols = test.interaction(factors=interaction_cols1,
                               pairwise=False,
                               max_factors=1000,
                               min_occurrence=100)
train = train.cbind(train_cols)                              #Append pairwise columns to H2OFrames
valid = valid.cbind(valid_cols)
test = test.cbind(test_cols)



In [ ]:
best_gbm.train(list(set(train.col_names) - set(['readmitted','diag_1', 'diag_2', 'diag_3'])), covtype_y,
                training_frame=train,
                validation_frame=valid)
best_gbm

In [ ]:
best_gbm.varimp_plot()

We see a further decrease in the validation set error.

In [ ]:
medicines = np.array(best_gbm.varimp()[-20:])

In [ ]:
medicines=medicines[:, 0]

In [ ]:
best_gbm.train(list(set(train.col_names) - set(['readmitted','diag_1', 'diag_2', 'diag_3']) - set(medicines)), covtype_y,
                training_frame=train,
                validation_frame=valid)
best_gbm

Removing these essentially useless features further improves our model's performance

In [ ]:
removed_features = set(['readmitted','diag_1', 'diag_2', 'diag_3'])
removed_features = removed_features.union(set(medicines))

In [ ]:
best_gbm.train(list(set(train.col_names) - set(['readmitted','diag_1', 'diag_2', 'diag_3'])), covtype_y,
                training_frame=train,
                validation_frame=valid)


In [ ]:
best_gbm

In [ ]:
best_gbm.varimp_plot()

Let's retrain the random forest aswell.

In [ ]:
best_rf.train(list(set(train.col_names) - set(['readmitted','diag_1', 'diag_2', 'diag_3']) - set(medicines)), covtype_y,
                training_frame=train,
                validation_frame=valid)


In [ ]:
pred_gbm = best_gbm.predict(valid)
pred_rf = best_rf.predict(valid)


In [ ]:
for i in range(0,21):
    pred = i/20*pred_gbm + (1-i/20)*pred_rf
    pred1= pred.as_data_frame()
    p1 = pred1.iloc[:, 1:]
    td.iloc[td.index[p1['NO'] == np.amax(p1.values, axis=1)],-1] = 'NO'
    td.iloc[td.index[p1['>30'] == np.amax(p1.values, axis=1)],-1] = '>30'
    td.iloc[td.index[p1['<30'] == np.amax(p1.values, axis=1)],-1] = '<30'
    print((((td2 == td)['readmitted']).sum())/(td2['readmitted'].count()), i)

In [ ]:
pred_gbm = best_gbm.predict(test)
pred_rf = best_rf.predict(test)

In [ ]:
f1.count(),f1.sum()

In [ ]:
f2.count(),f2.sum()